In [ ]:
%load_ext autoreload
%autoreload 2

print("Autoreload extension loaded. Code changes will be automatically reloaded.")


In [ ]:
# note: Based on https://github.com/jbloomAus/SAELens/blob/main/tutorials/using_an_sae_as_a_steering_vector.ipynb

# Install / Setup

In [ ]:
try:
  # for google colab users
    import google.colab # type: ignore
    from google.colab import output
    COLAB = True
    %pip install sae-lens transformer-lens
except:
  # for local setup
    COLAB = False
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")


# Imports for displaying vis in Colab / notebook
import webbrowser
import http.server
import socketserver
import threading
import functools

PORT = 8000

# general imports
import os
import torch
from tqdm import tqdm
import plotly.express as px

In [ ]:
import torch
import transformer_lens


# TODO(bschoen): Just start using `transformer_lens.utils.get_device` from now on
def get_best_available_torch_device() -> torch.device:
    return transformer_lens.utils.get_device()

In [ ]:
from jaxtyping import Float, Int

In [ ]:
from gpt_from_scratch import python_utils

In [ ]:
# imports from https://github.com/jbloomAus/SAELens/blob/main/tutorials/tutorial_2_0.ipynb
import os
import dataclasses

import torch
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import tabulate

In [ ]:
# disable autograd, as we're focused on inference and this save us a lot of speed, memory, and annoying boilerplate
torch.set_grad_enabled(False)

In [ ]:
def display_vis_inline(filename: str, height: int = 850):
    """
    Displays the HTML files in Colab. Uses global `PORT` variable defined in prev cell, so that each
    vis has a unique port without having to define a port within the function.
    """
    if not (COLAB):
        webbrowser.open(filename)

    else:
        global PORT

        def serve(directory):
            os.chdir(directory)

            # Create a handler for serving files
            handler = http.server.SimpleHTTPRequestHandler

            # Create a socket server with the handler
            with socketserver.TCPServer(("", PORT), handler) as httpd:
                print(f"Serving files from {directory} on port {PORT}")
                httpd.serve_forever()

        thread = threading.Thread(target=serve, args=("/content",))
        thread.start()

        output.serve_kernel_port_as_iframe(
            PORT, path=f"/{filename}", height=height, cache_in_notebook=True
        )

        PORT += 1

# Load your model and SAE

In [ ]:
device = get_best_available_torch_device()

print(f"Using device: {device}")

In [ ]:
model = transformer_lens.HookedTransformer.from_pretrained(
    "gemma-2b",
    device=device,
)